In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV,ShuffleSplit, cross_validate
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,OrdinalEncoder,MaxAbsScaler,PolynomialFeatures,StandardScaler,QuantileTransformer,RobustScaler
from sklearn.linear_model import LinearRegression, SGDRegressor,Ridge,Lasso,RidgeCV,LassoCV,ElasticNetCV
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest, VarianceThreshold, mutual_info_regression,f_regression
from sklearn.compose import TransformedTargetRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error,accuracy_score,r2_score,explained_variance_score,mean_absolute_error,median_absolute_error
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,HistGradientBoostingRegressor,StackingRegressor,ExtraTreesRegressor,VotingRegressor
from sklearn.svm import SVR,NuSVR,LinearSVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import statistics
import numpy as np

In [ ]:
# train_path = "/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv"
# train_data = pd.read_csv(train_path)

# y_train = train_data["total_amount"]


# dummy_regr = DummyRegressor(strategy="mean")
# dummy_regr.fit(x_train, y_train)
# dummy_regr.predict(x_train)

# dummy_regr.score(x_train,y_train)

# test_data = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv")

# prediction = dummy_regr.predict(test_data)

# submission = pd.DataFrame(columns=['ID','total_amount'])

# submission['ID'] = [i for i in range(1,len(prediction)+1) ]

# submission['total_amount'] = prediction
# submission.to_csv('submission.csv',index=False)

In [ ]:
# READING THE TRAINING DATA

x_train = pd.read_csv("/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv")
data = x_train
y_train = data[["total_amount"]]
print(x_train.info())
x_train.head()




<H2>PLOTTING HISTOGRAMS OF SOME COLUMNS FOR ANALYSIS</H2>

In [ ]:
x_train['total_amount'].plot(kind='hist', bins=200, rwidth=0.9, color='#600000')
plt.show()

In [ ]:
x_train['passenger_count'].plot(kind = 'hist',bins = 20,rwidth = 0.9)
value_counts = x_train['passenger_count'].value_counts()
for index, value in value_counts.items():
    plt.text(index, value, str(value), ha='center', va='bottom')

plt.show()

In [ ]:
#BOX PLOT FOR VISUALLY DETECTING THE OUTLIERS
x_train['trip_distance'].plot(kind='hist', bins=500, rwidth=0.9, color='#600000')
plt.show()
plt.figure(figsize=(8, 6))
plt.boxplot(x_train['trip_distance']) 
plt.xlabel('Trip Distance')
plt.title('Box Plot of Trip Distance')
plt.show()
x_train['trip_distance'].describe()


In [ ]:
#REMOVING THE OUTLIERS
mask_trip_distance = x_train['trip_distance'] <= 500
x_train = x_train[mask_trip_distance]
x_train['trip_distance'].describe()



In [ ]:
x_train['extra'].plot(kind='hist')
plt.show()
x_train['extra'].describe()

In [ ]:
x_train['tip_amount'].plot(kind='hist')
plt.show()
# x_train['tip_amount'].descirbe()

In [ ]:
x_train['tolls_amount'].plot(kind='hist')
plt.show()
x_train['tolls_amount'].describe()

In [ ]:
x_train['improvement_surcharge'].plot(kind='hist')
plt.show()
x_train['improvement_surcharge'].describe()

In [ ]:
x_train['congestion_surcharge'].plot(kind='hist')
plt.show()
x_train['congestion_surcharge'].describe()

In [ ]:
x_train['Airport_fee'].plot(kind='hist')
plt.show()
x_train['Airport_fee'].describe()

In [ ]:


# x_train.isna().sum()
# # This is for handling all the NAN values:
# ### data cleaning  ###

# x_train.describe().transpose()
# x_train['passenger_count'].plot(kind = 'hist',bins = 20,rwidth = 0.9)
# plt.show()

# random_numbers = np.random.randint(1, 3, size=len(x_train))
# x_train['passenger_count'].fillna(pd.Series(random_numbers), inplace=True)

# # Dealing with Outliers 

# def replace_invalid_values(value):
#     if value > 8:
#         return np.random.randint(1, 5)
#     return value
# x_train['passenger_count'] = x_train['passenger_count'].apply(replace_invalid_values)

# x_train['passenger_count'].plot(kind = 'hist',bins = 10,rwidth = 0.9)
# plt.show()


# #for trip distance

# print(x_train['trip_distance'].describe())
# print(len(x_train))
# # mask = (x_train['total_amount'] == 0) & (x_train['trip_distance'] == 0)
# x_train = x_train[(x_train['total_amount'] != 0) & (x_train['trip_distance'] != 0)]
# print(len(x_train))
# # print(y_train['total_amount'].describe())
# x_train['total_amount'].describe()
# # x_train['fare_per_mile'] = data.apply(lambda row: 0 if row['total_amount'] == 0 else abs(row['trip_distance'] / row['total_amount']), axis=1)

# # print(x_train['fare_per_mile'].describe())
# x_train['fare_per_mile'] = np.where((x_train['total_amount'] > 0) & (x_train['trip_distance'] > 0),
#                                     x_train['total_amount'] / x_train['trip_distance'],
#                                     np.nan)
# x_train['fare_per_mile'].describe()
# # print(x_train['fare_per_mile'].isna().sum())
# # x_train['total_amount'].describe()
# import matplotlib.pyplot as plt
# import pandas as pd

# # Assuming 'x_train' is your DataFrame
# # Replace 'x_train' with your actual DataFrame

# # Create bins
# bins = [10*i for i in range(21)]
# # bins = [ 0, 2000, 4000, 6000, 8000,10000, 12000, 14000, 16000, 18000, 20000, 22000, 24000, 26000, 28000, 30000, 32000, 34000, 36000, 38000, 40000, 42000, 44000, 46000, 48000, 49100]

# # Assign the 'fare_per_mile' values to bins
# x_train['fare_per_mile_bin'] = pd.cut(x_train['fare_per_mile'], bins=bins)

# # Plot the histogram with count labels
# plt.figure(figsize=(10, 6))
# ax = x_train['fare_per_mile_bin'].value_counts().sort_index().plot(kind='bar', edgecolor='black')

# # Display count labels over each bar
# for p in ax.patches:
#     ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='baseline', xytext=(0, 5), textcoords='offset points')

# # Set plot labels and title
# plt.xlabel('fare_per_mile Bins')
# plt.ylabel('Count')
# plt.title('Distribution of fare_per_mile')

# # Show the plot
# plt.show()

# # # Assuming 'x_train' is your DataFrame
# # # Replace 'x_train' with your actual DataFrame

# # # Drop rows with NaN values in 'fare_per_mile_bin'
# # x_train_cleaned = x_train.dropna(subset=['fare_per_mile_bin'])

# # # Create a boolean mask for the specified bin range
# # bin_range = pd.Interval(2000, 4000, closed='right')
# # mask_bin_2000_4000 = x_train_cleaned['fare_per_mile_bin'].apply(lambda x: x in bin_range)

# # # Filter rows based on the mask
# # rows_bin_2000_4000 = x_train_cleaned[mask_bin_2000_4000]

# # # Print the selected rows
# # print(rows_bin_2000_4000)

# x_train[(x_train['total_amount'] <= 0) & (x_train['trip_distance'] > 50)]
# x_train[(x_train['total_amount'] <= 0) & (x_train['trip_distance'] > 0) & (x_train['trip_distance'] <1)]

# x_train[x_train['trip_distance'] == 0]
# data['total_amount'].describe()
# def update_total_amount(row):
#     # if row['total_amount'] == 0 and row['trip_distance'] < 300:
#     if row['total_amount'] <= 0:
#         return row['trip_distance'] * 16
#     return row['total_amount']
# x_train['total_amount'] = x_train.apply(update_total_amount, axis=1)
# x_train['total_amount'].describe()
# x_train[x_train['total_amount'] == 0]
# x_train['trip_distance'].describe()

# def update_trip_distance(row):

#     if row['trip_distance'] == 0 or row['trip_distance'] > 150 or row['total_amount'] / row['trip_distance'] > 200:
#         return row['total_amount'] / 16
#     else:
#         return row['trip_distance']
# x_train['trip_distance'] = x_train.apply(update_trip_distance, axis=1)


# print(x_train['trip_distance'].describe())



# import matplotlib.pyplot as plt
# import pandas as pd

# # Assuming 'x_train' is your DataFrame
# # Replace 'x_train' with your actual DataFrame

# # Create bins
# bins = [10*i for i in range(21)]
# # bins = [ 0, 2000, 4000, 6000, 8000,10000, 12000, 14000, 16000, 18000, 20000, 22000, 24000, 26000, 28000, 30000, 32000, 34000, 36000, 38000, 40000, 42000, 44000, 46000, 48000, 49100]

# # Assign the 'fare_per_mile' values to bins
# x_train['trip_distance_bin'] = pd.cut(x_train['trip_distance'], bins=bins)

# # Plot the histogram with count labels
# plt.figure(figsize=(10, 6))
# ax = x_train['trip_distance_bin'].value_counts().sort_index().plot(kind='bar', edgecolor='black')

# # Display count labels over each bar
# for p in ax.patches:
#     ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()),
#                 ha='center', va='baseline', xytext=(0, 5), textcoords='offset points')

# # Set plot labels and title
# plt.xlabel('trip_distance Bins')
# plt.ylabel('Count')
# plt.title('Distribution of trip_distance_bin')

# # Show the plot
# plt.show()

# # for RateCodeID
# choices = [1, 2, 3, 4, 5, 99]
# x_train['RatecodeID'].fillna(random.choice(choices), inplace=True)
# x_train['RatecodeID'].isna().sum()
# x_train.isna().sum()
# x_train = x_train.drop(columns=['fare_per_mile_bin','trip_distance_bin'])
# x_train.isna().sum()
# x_train['extra'] = x_train['extra'].abs()
# x_train['tolls_amount'] = x_train['tolls_amount'].abs()
# x_train['improvement_surcharge'] = x_train['improvement_surcharge'].abs()
# x_train['congestion_surcharge'] = x_train['congestion_surcharge'].abs()
# x_train['Airport_fee'] = x_train['Airport_fee'].abs()

# def impute_congestion_charge(row):
#     if pd.isna(row['congestion_surcharge']):
#         return random.choice([0, 2.5])
#     else:
#         return row['congestion_surcharge']

# x_train['congestion_surcharge'] = x_train.apply(impute_congestion_charge, axis=1)
# def impute_congestion_charge(row):
#     if pd.isna(row['Airport_fee']):
#         return random.choice([0, 1.75])
#     else:
#         return row['Airport_fee']

# # x_train['Airport_fee'] = x_train.apply(impute_congestion_charge, axis=1)
# x_train.isna().sum()
# # # Assuming 'x_train' is your DataFrame
# # initial_row_count = len(x_train)

# # # Drop rows with NaN values in 'trip_distance' column
# # x_train.dropna(subset=['trip_distance'], inplace=True)

# # # Calculate the number of rows dropped
# # rows_dropped = initial_row_count - len(x_train)

# # # Display the number of rows dropped
# # print(f'{rows_dropped} rows were dropped.')

# # import matplotlib.pyplot as plt


# # data2['total_amount'] = data2['total_amount'].apply(abs)

# # plt.figure(figsize=(8, 6))  # Optional: Set the figure size

# # # Create a scatter plot
# # plt.scatter(data['trip_distance'], data['total_amount'], alpha=0.5)
# # plt.xlabel('trip_distance')
# # plt.ylabel('total_amount')
# # plt.title('Scatter Plot between Column 1 and Column 2')
# # plt.xlim(0.5, 100)
# # plt.grid(True)
# # plt.show()
# #cleaning the date and time column

# x_train['tpep_pickup_datetime'] = pd.to_datetime(x_train['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
# x_train['tpep_dropoff_datetime'] = pd.to_datetime(x_train['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')

# invalid_entries = x_train[x_train['tpep_pickup_datetime'] > x_train['tpep_dropoff_datetime']]
# for index in invalid_entries.index:
#     temp = x_train.at[index, 'tpep_pickup_datetime']
#     x_train.at[index, 'tpep_pickup_datetime'] = x_train.at[index, 'tpep_dropoff_datetime']
#     x_train.at[index, 'tpep_dropoff_datetime'] = temp
# # invalid_entries = x_train[x_train['tpep_pickup_datetime'] > x_train['tpep_dropoff_datetime']]

# print(invalid_entries)


# x_train.info()
# x_train['journey_duration'] = x_train['tpep_dropoff_datetime'] - x_train['tpep_pickup_datetime']

# # Filter entries where journey duration is greater than one day
# long_duration_entries = x_train[x_train['journey_duration'] > pd.Timedelta('1 day')]

# long_duration_entries



# (x_train.sort_values(by=['journey_duration'], ascending=False)).head()
# # print(long_duration_entries[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance']])

# x_train = x_train[~((x_train['journey_duration'] > pd.Timedelta('1 day')) & (x_train['trip_distance'] < 5))]
# x_train.reset_index(drop=True, inplace=True)

# long_duration_entries = x_train[x_train['journey_duration'] > pd.Timedelta('1 day')]
# x_train['travel_time'] = (((pd.to_datetime(x_train['tpep_dropoff_datetime']))- (pd.to_datetime(x_train['tpep_pickup_datetime']))).dt.total_seconds()/60)
# x_train['travel_time']=x_train['travel_time'].abs()

# def hr_to_time(hr):
#     if hr >= 0 and hr < 6:
#         return 0
#     if hr >=6 and hr <= 11:
#         return 1
#     elif hr >=11 and hr <= 17:
#         return 2
#     elif hr >=17 and hr <= 23:
#         return 3
#     else:
#         return 4
# x_train['pickup_hr'] = pd.to_datetime(x_train['tpep_pickup_datetime']).dt.hour
# x_train['pickup_time'] = x_train['pickup_hr'].apply(hr_to_time)

# x_train['dropoff_hr'] = pd.to_datetime(x_train['tpep_dropoff_datetime']).dt.hour
# x_train['dropoff_time'] = x_train['dropoff_hr'].apply(hr_to_time)


# x_train['date'] = pd.to_datetime(x_train['tpep_pickup_datetime']).dt.day
# # x_train['date_number'] = x_train['date'].apply(date_to_number)


# x_train['day_name'] = pd.to_datetime(x_train['tpep_pickup_datetime']).dt.day_name()
# # x_train['day_name_number'] = x_train['day_name'].apply(dayname_to_number)
# x_train = pd.get_dummies(x_train, columns=['day_name'])


# def month_to_number(m):
#     if m == 6:
#         return 0
#     elif m == 7 :
#         return 1
#     else:
#         return 2
    
# x_train['month'] = pd.to_datetime(x_train['tpep_pickup_datetime']).dt.month
# # x_train['month_number'] = x_train['month'].apply(month_to_number)
# x_train.head()
# x_train.columns

# x_train['store_and_fwd_flag'].fillna('N',inplace=True)

# def flag_to_number(f):
#     if f == 'N':
#         return 0
#     elif f == 'Y' :
#         return 1
#     else:
#         return 2
# x_train['flag_number'] = x_train['store_and_fwd_flag'].apply(flag_to_number)
# x_train['payment_type'].replace('unknown', 'Credit Card', inplace = True)
# def payment_to_number(p):
#     if p == 'Credit Card':
#         return 0
#     elif p == 'Cash':
#         return 1
#     elif p == 'Wallet':
#         return 2
#     else:
#         return 3

# x_train['payment_number'] = x_train['payment_type'].apply(payment_to_number)


# columns_to_exclude = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'payment_type','store_and_fwd_flag','journey_duration','fare_per_mile', 'pickup_hr', 'dropoff_hr', 'date', 'month']
# X = x_train.drop(['total_amount'] + columns_to_exclude, axis=1)  # Exclude 'total_amount' and specified columns
# y = x_train['total_amount']
# X = X.astype('float64')
# print(len(X.columns))
# # X.head()

*******************************************************************************

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


In [ ]:
# param_grid = {
#     'n_estimators': [10, 50, 100],
#     'max_depth': [None, 10, 20],
# #     'min_samples_split': [2, 5, 10],
# #     'min_samples_leaf': [1, 2, 4]
# #     'pca__n_components':[2,4,6,8,10]
# }


# rf_regressor = RandomForestRegressor(random_state=42)


# grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=4, scoring='r2',n_jobs=-1)
# grid_search.fit(X_train, y_train)


# best_params = grid_search.best_params_

In [ ]:
################################################################################


In [ ]:
x_train = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv')

<H3>HANDLING THE DATE AND TIME COLUMN</H3>
HERE THE PICKUP AND DROP TIME COLUMN IS CONVERTED TO DURATION COLUMN, CREATING A NEW FEATURE WHICH IS MORE USEFUL

In [ ]:
x_train['tpep_pickup_datetime'] = pd.to_datetime(x_train['tpep_pickup_datetime'])
x_train['tpep_dropoff_datetime'] = pd.to_datetime(x_train['tpep_dropoff_datetime'])


x_train['duration'] = (x_train['tpep_dropoff_datetime'] - x_train['tpep_pickup_datetime']).apply(lambda x: x.total_seconds())


x_train['duration'] = x_train['duration'].abs()
# x_train['duration'].head(10)
x_train = x_train.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis = 1)


ENCODING THE CATEGORICAL COLUMNS

In [ ]:
x_train = pd.get_dummies(x_train, columns=['store_and_fwd_flag', 'payment_type'], 
                         prefix=['store_and_fwd', 'payment_type'])

In [ ]:
# x_train['extra'] = x_train['extra'].abs()
# x_train['tolls_amount'] = x_train['tolls_amount'].abs()
# x_train['improvement_surcharge'] = x_train['improvement_surcharge'].abs()
# x_train['congestion_surcharge'] = x_train['congestion_surcharge'].abs()
# x_train['Airport_fee'] = x_train['Airport_fee'].abs()
x_train = x_train.astype('float64') #CONVERTING ALL THE COLUMNS TO SAME TYPE
x_train.info()
x_train.isna().sum()


In [ ]:
ii = IterativeImputer(max_iter = 15)
x_train_imputed = ii.fit_transform(x_train)
x_train = pd.DataFrame(x_train_imputed, columns=x_train.columns)
#HEAT MAP FOR CHEKCING THE CORRELATION BETWEEN COLUMNS TO SEE RELEVANCE OF FEATURES.
corr = x_train.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap of x_train')
plt.show()
X = x_train.drop('total_amount',axis = 1)
Y = x_train['total_amount']
print(X.isna().sum())
print(Y.isna().sum())


In [ ]:
y_train = x_train['total_amount']
x_train = x_train.drop('total_amount',axis=1)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


RANDOM FOREST

In [ ]:
rf = RandomForestRegressor()
rf_ = RandomForestRegressor()
rf.fit(x_train,y_train)
rf_.fit(X,Y)
y_pred_rf = rf.predict(x_test)
r2 = r2_score(y_test,y_pred_rf)
rme = np.sqrt(mean_squared_error(y_test,y_pred_rf))
print("R2 Score for the Random Forest Regressor is:",r2)
print("RMSE with Random Forest Regressor is:",rme)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_rf, alpha=0.5)
plt.title('Actual vs Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

BAGGING REGRESSOR

In [ ]:
br = BaggingRegressor()
br_ = BaggingRegressor()
br.fit(x_train,y_train)
br_.fit(X,Y)
y_pred_br = br.predict(x_test)
r2 = r2_score(y_test,y_pred_br)
rme = np.sqrt(mean_squared_error(y_test,y_pred_br))
print("R2 Score for the Random Forest Regressor is:",r2)
print("RMSE with Random Forest Regressor is:",rme)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_br, alpha=0.5)
plt.title('Actual vs Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

GRID SEARCH ON EXTRATREE REGRESSOR

In [ ]:
# #USING PIPELINE AND GRID SEARCH CV
pipeline = Pipeline([ ('ss', StandardScaler()), ('poly',PolynomialFeatures(degree= 2)),('pca',PCA()),('r',ExtraTreesRegressor(n_jobs = -1)) ])
parameters = dict(r__n_estimators=[100,150],pca__n_components=[5,10])
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(x_train,y_train)
grid_search_pred = grid_search.predict(x_test)
r2 = r2_score(y_test,grid_search_pred)
rme = np.sqrt(mean_squared_error(y_test,grid_search_pred))
best_params = grid_search.best_params_
print("BEST PARAMETERS ARE:", best_params)
print("R2 Score for the Random Forest Regressor is:",r2)
print("RMSE with Random Forest Regressor is:",rme)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, grid_search_pred, alpha=0.5)
plt.title('Actual vs Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()



<h1>ACTUAL TESTING DATA</h1>
***FOR SUBMISSION***

In [ ]:
x_test = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv')

In [ ]:
x_test['tpep_pickup_datetime'] = pd.to_datetime(x_test['tpep_pickup_datetime'])
x_test['tpep_dropoff_datetime'] = pd.to_datetime(x_test['tpep_dropoff_datetime'])

x_test['duration'] = (x_test['tpep_dropoff_datetime'] - x_test['tpep_pickup_datetime']).apply(lambda x: x.total_seconds())

x_test['duration'] = x_test['duration'].abs()
x_test = x_test.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis = 1)


In [ ]:
# x_test['store_and_fwd_flag'].fillna('N',inplace=True)
# x_test['payment_type'].fillna('Credit Card',inplace=True)
# x_test['passenger_count'] = x_test['passenger_count'].apply(replace_invalid_values)



In [ ]:
x_test = pd.get_dummies(x_test, columns=['store_and_fwd_flag', 'payment_type'], 
                         prefix=['store_and_fwd', 'payment_type'])

In [ ]:
# x_test['extra'] = x_test['extra'].abs()
# x_test['tolls_amount'] = x_test['tolls_amount'].abs()
# x_test['improvement_surcharge'] = x_test['improvement_surcharge'].abs()
# x_test['congestion_surcharge'] = x_test['congestion_surcharge'].abs()
# x_test['Airport_fee'] = x_test['Airport_fee'].abs()
x_test = x_test.astype('float64')
x_test.info()
x_test.isna().sum()

In [ ]:
x_test_imputed = ii.fit_transform(x_test)
x_test = pd.DataFrame(x_test_imputed, columns=x_test.columns)
x_test.info()


In [ ]:
ypredict = rf_.predict(x_test) #score was = 0.93289
# ypredict = br_.predict(x_test) score was = 0.92633

print(len(ypredict))


submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(ypredict)+1)]
submission["total_amount"] = pd.DataFrame(ypredict)
submission.to_csv('submission.csv',index=False)
print("Done")